In [6]:
from src.vgg import vgg19
from src.utils import count_parameters, load_weights

model = vgg19()

device = "cuda:7"
model.to(device)

load_weights(model, "out/accelerating_training_large/best_model.pt")

In [7]:
model

vgg(
  (feature): Sequential(
    (0): Conv2d(3, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (4): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (8): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (11): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (12): ReLU(inplace=True)
    (13): MaxPool2d(kernel_size=2, stride=2, padding=0, d

In [8]:
num_params, top = count_parameters(model, return_top=True)
print("num params:", num_params, "top:", top)

num params: 174679012 top: [(37748736, 'feature.43.weight'), (37748736, 'feature.46.weight'), (37748736, 'feature.49.weight'), (18874368, 'feature.40.weight'), (9437184, 'feature.30.weight')]


In [9]:
from src.dataloader import load_cifar

_, eval_loader = load_cifar(train_batch_size=64, eval_batch_size=64)

Files already downloaded and verified
Files already downloaded and verified


In [10]:
import torch as th
import numpy as np
from sklearn.metrics import accuracy_score

def eval(model, device, eval_loader):
    labels_true = []
    labels_pred = []
    for step, (data, label) in enumerate(eval_loader, start=1):
        data = data.to(device)

        with th.no_grad():
            label_pred = model(data).argmax(axis=-1).squeeze()

        labels_true.append(label.numpy())
        labels_pred.append(label_pred.cpu().numpy())
        
    score = accuracy_score(np.concatenate(labels_true), np.concatenate(labels_pred))
    return score

In [11]:
eval(model, device, eval_loader)

0.5718

In [12]:
from src.decomposition import tucker_decompose_model
import tensorly as tl

tl.set_backend('pytorch')
model = tucker_decompose_model(model, linear_max_rank=-1, conv_max_rank=-1)

/home/mavrinti/.venv/lib/python3.10/site-packages/tensorly/decomposition/_tucker.py:167: Warning: Given only one int for 'rank' instead of a list of 2 modes. Using this rank for all modes.
  warnings.warn(message, Warning)


In [13]:
model

vgg(
  (feature): Sequential(
    (0): Conv2d(3, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (4): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (8): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (11): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (12): ReLU(inplace=True)
    (13): MaxPool2d(kernel_size=2, stride=2, padding=0, d

In [14]:
num_params, top = count_parameters(model, return_top=True)
print("num params:", num_params, "top:", top)

num params: 15881188 top: [(4718592, 'feature.27.weight'), (2359296, 'feature.17.weight'), (2359296, 'feature.20.weight'), (2359296, 'feature.23.weight'), (1179648, 'feature.14.weight')]


In [15]:
eval(model, device, eval_loader)

0.5407

---